# Data ADS

In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials

import matplotlib.pyplot as plt

In [2]:
# 1. Đọc danh sách ASIN từ file txt
with open('ASIN.txt', 'r') as f:
    asin_list = [line.strip() for line in f.readlines()]

# 2. Tạo list chứa các file .xlsx trong thư mục Data
folder_path = 'Data_ADS'
files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

# 3. Đọc từng file, xử lý rồi append vào list
all_data = []

for file in files:
    # Trích xuất ngày từ tên file
    date_str = file.split('_')[1].replace('.xlsx', '')  # "20250713"
    date_obj = datetime.strptime(date_str, "%Y%m%d")     # datetime object
    df = pd.read_excel(os.path.join(folder_path, file))
    
    # Thêm cột date định dạng chuẩn
    df['date'] = date_obj
    
    # Tạo cột ASIN từ 10 ký tự đầu tiên của cột Portfolio
    df['ASIN'] = df['Portfolio'].astype(str).str[:10]
    
    all_data.append(df)

# 4. Gộp tất cả thành một DataFrame
merged_df = pd.concat(all_data, ignore_index=True)

# 5. Lọc chỉ giữ lại các dòng có ASIN nằm trong danh sách
filtered_df = merged_df[merged_df['ASIN'].isin(asin_list)]

#6. Di chuuyển cột ASIN lên đầu, sau Campaign type	
filtered_df = filtered_df[['Campaign type', 'ASIN'] + [col for col in filtered_df.columns if col not in ['Campaign type', 'ASIN']]]

# ✅ Kết quả cuối cùng
filtered_df.head()

,Campaign type,ASIN,Campaign,Status,Country,Profile,Portfolio,Target type,Daily Budget,Current Budget,...,CPA,Sales Same SKU,Sales Other SKU,Orders Same SKU,Orders Other SKU,Units Same SKU,Units Other SKU,date,Top-of-search IS,Avg.time in Budget
27,sponsoredProducts,B08R8R2LQF,B08R8R2LQF_20oz_May the forties black_40th bir...,Out Of Budget,US,NewEleven,B08R8R2LQF_TUMBLER 20_MAY THE FORTIES BE WITH ...,manual,$10.00,$0.00,...,23.85,22.98,0.00,1,--,1,--,2025-07-27,NaN,NaN
31,sponsoredProducts,B0DH86X93V,B0DH86X93V_20oz_vintage 1985 weird navy_40th b...,Delivering,US,NewEleven,B0DH86X93V_TUMBLER 20_VINTAGE 1985 WEIRD BEING...,manual,$24.00,$0.00,...,7.79,19.98,42.96,1,2,1,2,2025-07-27,NaN,NaN
56,sponsoredProducts,B0F2T4JDF7,B0F2T4JDF7_Card_Youre awesome mom_birthday car...,Delivering,US,NewEleven,B0F2T4JDF7_CARD_YOU'RE AN AWESOME MOM_TRINH,manual,$10.00,$0.00,...,4.80,39.96,0.00,4,--,4,--,2025-07-27,NaN,NaN
60,sponsoredProducts,B092HNBJT6,B092HNBJT6_20oz_Coworker black_auto 15h40 (max...,Out Of Budget,US,NewEleven,B092HNBJT6_TUMBLER 20_COWORKER BLACK_TRINH,auto,$10.00,$0.00,...,6.30,0.00,59.94,0,3,0,3,2025-07-27,NaN,NaN
72,sponsoredProducts,B09XXP5V2W,"B09XXP5V2W_20oz_Quitter black_retirement gift_b,p",Out Of Budget,US,NewEleven,B09XXP5V2W_TUMBLER 20_QUITTER BLACK_TRINH,manual,$10.00,$0.00,...,0.00,0.00,0.00,0,--,0,--,2025-07-27,NaN,NaN


In [3]:
scopes = ["https://www.googleapis.com/auth/spreadsheets", 
          "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file("/Users/ppcintern/Downloads/new_credential.json", scopes=scopes)
client = gspread.authorize(creds)

# Mở Google Sheet
sheet_id = "1n2Ug3joJwsuuSsrx6T92Zi_dihht7KzIebRTGpKnehc"

# Mở file Google Sheet (Spreadsheet object)
spreadsheet = client.open_by_key(sheet_id)
sheet1 = client.open_by_key(sheet_id).worksheet("DATA_XN_ADS")

set_with_dataframe(sheet1, filtered_df)

# Total GMV

In [15]:
import numpy as np
import pandas as pd
import os
from datetime import datetime
import re

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2.service_account import Credentials

import matplotlib.pyplot as plt

In [16]:
# 1. Đọc danh sách ASIN từ file txt
with open('ASIN.txt', 'r') as f:
    asin_list = [line.strip() for line in f.readlines()]

# 2. Tạo list chứa các file .csv trong thư mục Data
folder_path = 'Data_Total'
files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# 3. Đọc từng file, xử lý rồi append vào list
all_data = []

for file in files:
    # ✅ Dùng regex để tìm ngày theo pattern dd_mm_yyyy
    match = re.search(r'(\d{2}_\d{2}_\d{4})', file)
    
    if match:
        date_str = match.group(1)  # ví dụ: "20_07_2025"
        date_obj = datetime.strptime(date_str, "%d_%m_%Y")  # chuyển thành datetime object

        df = pd.read_csv(os.path.join(folder_path, file), sep=";")
        
        # Thêm cột 'Date' chuẩn hóa theo yyyy-mm-dd
        df['Date'] = date_obj.strftime('%Y-%m-%d')
        
        all_data.append(df)
    else:
        print(f"⚠️ Không tìm thấy ngày trong tên file: {file}")

# 4. Gộp tất cả thành một DataFrame
merged_df = pd.concat(all_data, ignore_index=True)

# ✅ 5. Chỉ giữ lại các dòng có ASIN nằm trong danh sách
merged_df = merged_df[merged_df['ASIN'].isin(asin_list)]

# 6. Ở cột Ads, Sales và Net profit, thay thế dấu phẩy bằng dấu chấm
merged_df['Ads'] = merged_df['Ads'].str.replace(',', '.').astype(float)
merged_df['Sales'] = merged_df['Sales'].str.replace(',', '.').astype(float)
merged_df['Net profit'] = merged_df['Net profit'].str.replace(',', '.').astype

In [17]:
merged_df.head(7)

,Product,ASIN,SKU,Units,Refunds,Sales,Promo,Ads,Sponsored products (PPC),Sponsored Display,...,Net profit,Estimated payout,Expenses,Margin,ROI,BSR,Real ACOS,Sessions,Unit Session Percentage,Date
12,NewEleven 40th Birthday Gifts For Men Women - ...,B08R8R2LQF,JC-JMFX-W7IY,15.0,1.0,344.70,NaN,-53.21,"-53,21",NaN,...,<bound method NDFrame.astype of 12 88.98\...,"152,13",NaN,"25,81","140,90",21 626,"15,44",43.0,"34,88",2025-07-24
24,NewEleven 40th Birthday Gifts For Men Women - ...,B0DH86X93V,Z0-L1YB-49DF,9.0,1.0,179.82,NaN,-19.21,"-19,21",NaN,...,<bound method NDFrame.astype of 12 88.98\...,"76,22",NaN,"25,92","157,41",2 731,"10,68",40.0,"22,50",2025-07-24
25,NewEleven Retirement Gifts For Men Women 2025 ...,B09XXP5V2W,FL-GKXG-L8RH,8.0,NaN,159.84,-1,-45.74,"-45,74",NaN,...,<bound method NDFrame.astype of 12 88.98\...,"53,66",NaN,"11,85","54,55",28 472,"28,62",64.0,"12,50",2025-07-24
28,NewEleven Gifts for Women Friendship - Funny G...,B0D9L8P1XL,1G-WJN6-34JP,8.0,1.0,143.84,NaN,-24.00,-24,NaN,...,<bound method NDFrame.astype of 12 88.98\...,"47,49",NaN,"7,71","30,47",160 443,"16,69",34.0,"23,53",2025-07-24
39,NewEleven Coworker Leaving Gifts - Gifts For C...,B0BRB6NJCT,MF-GYAU-GTQX,6.0,NaN,89.88,NaN,-12.86,"-12,86",NaN,...,<bound method NDFrame.astype of 12 88.98\...,"34,84",NaN,"22,61","139,94",17 123,"14,31",71.0,"8,45",2025-07-24
46,NewEleven Coworker Leaving Gifts - Farewell Gi...,B09ZYCD6GX,1Q-FSNP-VBED,5.0,NaN,99.90,-1,-13.25,"-13,25",NaN,...,<bound method NDFrame.astype of 12 88.98\...,"48,56",NaN,"27,49","130,14",24 920,"13,26",40.0,"12,50",2025-07-24
57,NewEleven Gifts For New Mom 2025 – Pregnancy G...,B0DCJM56W3,SG-SV6L-3ILW,4.0,NaN,51.96,NaN,-15.14,"-15,14",NaN,...,<bound method NDFrame.astype of 12 88.98\...,"8,26",NaN,"-1,27","-7,40",8 516,"29,14",39.0,"10,26",2025-07-24


In [18]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 186 entries, 12 to 2486
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Product                   186 non-null    object 
 1   ASIN                      186 non-null    object 
 2   SKU                       182 non-null    object 
 3   Units                     156 non-null    float64
 4   Refunds                   20 non-null     float64
 5   Sales                     156 non-null    float64
 6   Promo                     21 non-null     object 
 7   Ads                       124 non-null    float64
 8   Sponsored products (PPC)  124 non-null    object 
 9   Sponsored Display         0 non-null      float64
 10  Sponsored brands (HSA)    0 non-null      float64
 11  Sponsored Brands Video    0 non-null      float64
 12  Google ads                0 non-null      float64
 13  Facebook ads              0 non-null      float64
 14  % Refunds    

In [19]:
scopes = ["https://www.googleapis.com/auth/spreadsheets", 
          "https://www.googleapis.com/auth/drive"]
creds = Credentials.from_service_account_file("/Users/ppcintern/Downloads/new_credential.json", scopes=scopes)
client = gspread.authorize(creds)

# Mở Google Sheet
sheet_id = "1n2Ug3joJwsuuSsrx6T92Zi_dihht7KzIebRTGpKnehc"

# Mở file Google Sheet (Spreadsheet object)
spreadsheet = client.open_by_key(sheet_id)
sheet1 = client.open_by_key(sheet_id).worksheet("DATA_SB_TOTAL")

sheet1.clear()  # Xóa dữ liệu cũ trước khi ghi mới
set_with_dataframe(sheet1, merged_df)